## 모듈 불러오기

In [1]:
import numpy as np
from flask import Flask, request
from flask_sqlalchemy import SQLAlchemy

## DB 지정

In [2]:
app = Flask(__name__)

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///Server.db"
# DB를 지정 + 파일명을 지정, 작대기 3개면 상대경로 4개면 절대경로
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False # 꼭 필요하다고 합니다.

db = SQLAlchemy(app) # Flask app과 SQLAlchemy를 합체함.
db.init_app(app)

## table 정의

In [3]:
class MyTable(db.Model):
    # db.Model을 상속받으면 db.Column()이라는 메소드를 사용가능함.
    __tablename__ = "MyTable"
    id = db.Column(db.Integer, primary_key=True) # 꼭 있어야된다?
    
    def __repr__(self):
        return f"MyTable('{self.id}')"

In [4]:
class MyTable2(db.Model):
    # db.Model을 상속받으면 db.Column()이라는 메소드를 사용가능함.
    __tablename__ = "MyTable2"
    num = db.Column(db.Integer, primary_key=True) # 꼭 있어야된다?
    
    def __repr__(self):
        return f"MyTable('{self.num}')"

## DB 생성

In [5]:
db.create_all() # 이렇게 만든 테이블을 다 생성해봐라.

## 랜덤 서브셋 저장

In [6]:
all_set = [i for i in range(100)]
sub_set = np.random.choice(all_set, 50, replace=False) # 겹치지않게 고름

for val in sub_set:
    a = MyTable(id=int(val))
    db.session.add(a)

db.session.commit()

In [17]:
np.array(list(map(lambda x:x.id,MyTable.query.all())))

array([ 1,  2,  7, 10, 11, 17, 19, 21, 23, 25, 29, 32, 34, 36, 37, 38, 39,
       42, 45, 46, 47, 48, 49, 54, 56, 57, 62, 63, 64, 65, 67, 71, 72, 76,
       79, 80, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [12]:
np.save('./server_db.npy', np.array(list(map(lambda x:x.id,MyTable.query.all()))))

In [13]:
import numpy as np
import socket as sk
from threading import Thread, Lock
import time
import os

In [15]:
def get_filesize(path):
    size = os.path.getsize(path)
    return str(size)

def get_filedata(path):
    with open(path, 'rb') as f:
        return f.read()

In [18]:
HOST = ''
PORT = 12321

server = sk.socket(sk.AF_INET, sk.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(0)
with_client, addr = server.accept()
print('Connected.')

path = './server_db.npy'
while True:
    data = with_client.recv(1024).decode('utf-8')
    if data == 'get':
        with_client.send(get_filesize(path).encode('utf-8'))
        print('send filesize')
        time.sleep(0.1)
        with_client.send(get_filedata(path))
        print('send filedata')
        break

Connected.
